# Dataset

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [4]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [5]:
user_recos[33][0]

RecoPath(nodes=[RecoNode(type='user', entity_id=33), RecoNode(type='product', entity_id=2346), RecoNode(type='user', entity_id=2678), RecoNode(type='product', entity_id=1762)], rels=[RecoRel(in_node=RecoNode(type='user', entity_id=33), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=1762))])

# Explanation

In [6]:
import dotenv

dotenv.load_dotenv()

True

In [7]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [8]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    **{
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

template = """{context}

You are a tooltip explaining to {user} why {product} was recommended to them in a paragraph.
Be clear and concise, no need to greet the user."""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

/home/balfroim/.cache/pypoetry/virtualenvs/trail24-UZVMavSK-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/balfroim/.cache/huggingface/token
Login successful


In [9]:
from recommendation.registry_handler import RegistryHandler
from recommendation.explainers.llm_explainer import LLMExplainer


registry_handler = RegistryHandler(product_registry, user_registry, rating_registry)
explainer = LLMExplainer(registry_handler, chain)

In [10]:
path = user_recos[33][1]
# cot_explainer._prepare_input(path, ["name"])
continuation, _ = explainer.explain(path)

In [11]:
print(continuation)



Based on your viewing history, we recommend "Star Wars: Episode IV - A New Hope (1977)". This action movie has been rated highly by users with similar preferences to yours, specifically those who are in the same age range and gender as you. Additionally, this recommendation aligns with your interest in horror movies, as "Star Wars: Episode IV - A New Hope (1977)" is known for its thrilling and intense scenes.


In [12]:
continuation_without_name, _ = explainer.explain(path, ["name"])
print(continuation_without_name)



Based on your viewing history and ratings, we think you might enjoy "Product1359" because it is an action movie, which is a genre you have not yet explored. Users with similar demographics and viewing habits to yours have rated this product highly. We hope you find this recommendation helpful and enjoy the movie!
